## Hegselmann-Krause

I have written this Julia notebook for my essay on Modeling of Opinion Dynamics as part of my degree at the University of Cambridge (Part III of the Mathematical Tripos). 

This code is published under the creative commons licence BY-NC-SA (Attribution-NonCommercial-ShareAlike) and any feedback is highly appreciated.

In [ ]:
#import Pkg
#Pkg.add("Measures")

using Plots
using Random  
using LinearAlgebra
using LaTeXStrings
using Distributions
using DelimitedFiles

import Measures: Length, AbsoluteLength, Measure, BoundingBox, mm, cm, pt, width, height, w, h

const plt = Plots
const rnd = Random
const LA = LinearAlgebra

### Functions

#### Noise Functions

In [ ]:
# creates no noise, i.e. a noise vector of zeros
# the paraneter "param" is just to fit the overall structure of noise functions
function no_jumps(param, N)
    return zeros(N)    
end

In [ ]:
# creates zero-mean Gaussian random noise with stddev "strength"
function gaussian_random_jumps(strength, N)
    μ = 0
    σ = strength
    d = Normal(μ, σ) 
    ξ = rand(d,N)
    return ξ
end

In [ ]:
# creates uniformly distributed noise up to strength "bound", i.e. ξ ∈ [-bound, bound]
function uniform_bounded_jumps(bound, N)
    d = Uniform(-bound,bound)
    ξ = rand(d,N)
    return ξ
end

#### Simulation functions

In [ ]:
# for symmetric intervals call with ϵ_l = ϵ_r
# x_i and x_f have to be distinct vectors and not pointers to the same entity
# this method of HK_update is leading
function HK_update!(x_i, x_f, ϵ_l, ϵ_r; err_tol = 1e-9, noise_function = no_jumps, noise_param = 0.)
    #Number of agents
    N =  convert(Int,length(x_i))
    
    #Calculate interaction matrix
    I = Matrix{Float64}(LA.I, N, N)
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i[i]-x_i[j]) < ϵ_l+err_tol
                I[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i[i]-x_i[j]) < ϵ_r+err_tol
                I[i,j] = 1.0
            else
            end
        end
    end
    #normalize each row of the row-stochastic matrix
    for i in 1:N
        norm = sum(I[i,:])
        I[i,:] = I[i,:]./norm
    end
    
    #Generate noise if required
    ξ = noise_function(noise_param, N)


    #Update state vector
    for i in 1:N
        x_update = 0.

        #average over neigbouring opinions
        for j in 1:N
            if I[i,j]>err_tol
                x_update += I[i,j] * x_i[j] 
            else
            end
        end

        #add noise
        x_update += + ξ[i]
        
        #Adsorbing boundary condition
        if x_update > 1
            x_update = 1
        elseif x_update < 0
            x_update = 0
        else
        end
        
        #update final state vector
        x_f[i] = round(x_update, digits = 9)
    end
end

In [ ]:
#Simulate HK model to return full trajectory of opinion dynamics 
function HK_simulate(ϵ_l, ϵ_r;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed = 2310)
    #Initial Distribution
    rng = MersenneTwister(seed)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    trajectories = zeros(N,max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Run Simulation
    for i in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end

    return trajectories
end

In [ ]:
# Run simulation for ϵ_l = ϵ_r
function HK_simulate(ϵ;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed= 2310)
    trajectories = HK_simulate(ϵ, ϵ; N = N, max_iter = max_iter, initial = initial, noise_function = noise_function, noise_param = noise_param, seed= seed)
    return trajectories
end

In [ ]:
#Check symmetry of a state 
function check_sym(state; max = 1, min = 0, err_tol = 1e-9)
    N = length(state)
    n = floor(Int, N/2)
    println(N, " -> ", n)
    check = 0
    for i in 1:n
        if max - state[i] - state[N-i+1] < err_tol
        else
            check += 1
        end
    end
    println(check, " entries are not symmetric.")
end

In [ ]:
#Check if two state vectors have converged (up to err_tol)
function check_convergence(a::Vector,b::Vector; err_tol = 1e-9)
    #General check of input
    if length(a) != length(b)
        println("Not same length")
        return -1
    else
        #checking difference of vectors
        null = zeros(length(a)) .+ err_tol

        change = abs.(a .- b)
        check = change .< null
        if check == ones(length(check))
            return 1
        else
            return 0
        end
    end
end

#Wrapper function for checking convergence that returns some more information if not converged
function convergence(a,b; err_tol = 1e-9)
    if check_convergence(a,b) != 1
        println("Not converged!")
    
        letzt = (unique(b))
        vorletzt = (unique(a)) 
        if length(letzt) == length(vorletzt)
            #Plot to figure out reason why it has not converged yet
            plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=200, legend = :none)
            plt.plot!(xlabel = "Opinions", ylabel = "Change")
            plt.scatter!(vorletzt, letzt.-vorletzt)
        else
            print("Merging of clusters")
        end
        return -1
    else
        return 1
    end
end

In [ ]:
#Simulate HK model until it converges to return equilibrium state 
#return the last two states in simulation to allow for manual convergence checks
function HK_simulate_convergence(ϵ_l, ϵ_r;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0., seed = 2310)
    #Initial Distribution
    rng = MersenneTwister(seed)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for k in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)

        if check_convergence(x_old, x_new; err_tol = 1e-9) == 1
            #print("Converged at ", k)
            break
        elseif k != max_iter
            x_old = copy(x_new)
        else
        end
    end
    convergence(x_old, x_new; err_tol = 1e-9) != 1 ? println("not converged within max_iter") :
    return x_old,x_new
end

In [ ]:
# Simulation loop for specified initial condition array x_i
function HK_simulate(x_i::Array, ϵ_l, ϵ_r; max_iter = 20, noise_function = no_jumps, noise_param = 0., seed = 2310)

    #Initialize Simulation
    trajectories = zeros(length(x_i),max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for i in 1:max_iter
        HK_update!(x_old,x_new, ϵ_l, ϵ_r; noise_function = noise_function, noise_param = noise_param)
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end
    return trajectories
    
end

### Analysis of baseline model (symmetric confidence bounds)

#### Opinion Trajectories 

In [ ]:
N = 50
ϵ = 0.20
max_iter = 30
initial = "uni"

result = HK_simulate(ϵ; N = N, max_iter = max_iter, initial = initial, seed = 2310)

#for plotting purpose
max_iter = 10
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")#, xticks = [0,5,10])
for i in 1:N
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
end

#savefig("./Plots/HK-sym_N-$(N)_eps-$(round(ϵ,digits = 4))_init-dist-$(initial).png")

plt.plot!()

In [ ]:
# Sanity check
check_sym(result[:,end])

##### Changes

In [ ]:
N = 30
ϵ = 0.20
max_iter = 30
initial = "uni"

result = HK_simulate(ϵ; N = N, max_iter = max_iter, initial = initial, seed = 2310)

#for plotting purpose
max_iter = 10
plt.plot(layout = (2,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, ylabel = L"\textrm{Opinion}\;x_i(t)", xticks = [0,2,4,6,8,10])
plt.plot!(subplot = 2, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Change\;in\;Opinion}", xticks = [0,2,4,6,8,10])#, xticks = [0,5,10])
for i in 1:N
    if i == N
        plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:red, linealpha = 1, linesize = 0.3)
        plt.plot!(subplot = 2, 0.5:max_iter-0.5,result[i,2:max_iter+1] .- result[i,1:max_iter], color =:red, linealpha = 1, linesize = 0.3)
    elseif i == ceil(Int, N/2+1)
        plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:orange, linealpha = 1, linesize = 0.3)
        plt.plot!(subplot = 2, 0.5:max_iter-0.5,result[i,2:max_iter+1] .- result[i,1:max_iter], color =:orange, linealpha = 1, linesize = 0.3)

    else
        plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
        plt.plot!(subplot = 2, 0.5:max_iter-0.5,result[i,2:max_iter+1] .- result[i,1:max_iter], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
    end
end
plt.plot!(subplot = 2, xlim = [0,10])

#savefig("./Plots/HK-changes-in-x_N-$(N)_eps-$(round(ϵ,digits = 4))_init-dist-$(initial).png")

#### Phase diagram

In [ ]:
ϵ_list = 0.05:0.005:0.3

N =  1#30
max_iter = 300
initial = "uni"

plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Confidence\;Bound}\;ε", ylabel = L"\textrm{Final\;Opinion\;Cluster}")

number = zeros(length(ϵ_list))

for (i,e) in enumerate(ϵ_list)
    print(" - ",i, ": ")
    res = HK_simulate_convergence(e,e; N = N, max_iter = max_iter, initial = initial)
    
    #checking that clusters have actually converged
    convergence(res[1],res[2]) != 1 ? (println(e); break) : "" 
    
    #Save number of final clusters
    cluster = unique(res[2])
    number[i] = length(cluster)

    #Plot all positions of final clusters for each ε
    x = ones(length(cluster)) * e
    plt.scatter!(subplot = 1, x, cluster, color =:red, marker = :x)

    
end

plt.plot!()

In [ ]:
# plot line at lowest ε for which consensus is reached
# ind = findfirst(isequal(1.0),number)
# plt.vline!([ϵ_list[ind]], color = :grey, linestyle = :dot)

#plot line at lowest ε from which onwards only consensus will be reached
ind2 = findlast(x -> x != 1, number)
plt.vline!([ϵ_list[ind2+1]], color = :grey, linestyle = :dash)

#savefig("./Plots/HK-Phase-diag-sym-nonoise_N-$(N)_init-dist-$(initial).png")

plt.plot!()

#### Epsilon for Consensus

In [ ]:
ϵ_list = 0.18:0.005:0.24
N_list = 30 .* 2 .^collect(1:4)
initial = "uni"

consensus = zeros(length(N_list))
for (j,N) in enumerate(N_list)
    number = zeros(length(ϵ_list))
    for (i,e) in enumerate(ϵ_list)
        res = HK_simulate_convergence(e,e; N = N, max_iter = N, initial = initial)
        
        #checking that clusters have actually converged
        convergence(res[1],res[2]) != 1 ? (println(e); break) : "" 
        
        #Unique opinion clusters in final state
        cluster = unique(res[2])
        
        #Save number of final clusters
        number[i] = length(cluster)
    end
    #lowest ε for which consensus is reached
    # ind = findfirst(isequal(1.0),number)

    #lowest ε from which onwards only consensus will be reached
    ind = findlast(x -> x != 1, number)
    consensus[j] = ϵ_list[ind+1]
end


#plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
#plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Confidence\;Bound}\;ε", ylabel = L"\textrm{Final\;Opinion\;Cluster}")

plt.scatter(N_list, consensus)

#savefig("./Plots/HK-Phase-diag-sym-nonoise_N-$(N)_init-dist-$(initial).png")

plt.plot!()

##### All plots in one

In [ ]:
max = 40
ϵ_inv = 3:1:max

N = 300
max_iter = N
initial = "uni"
number = zeros(length(ϵ_inv))

plt.plot(layout = (3,1), size =(500,800), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Inverse\;Confidence\;Bound}\;1/ε", ylabel = L"\textrm{Final\;Opinion\;Cluster\;(rescaled)}")
plt.plot!(subplot = 2, legend = :none, xlabel = L"\textrm{Confidence\;Bound}\;ε", ylabel = L"\textrm{Final\;Opinion\;Cluster\;(shifted)}")
plt.plot!(subplot = 3, legend = :none, xlabel = L"\textrm{Inverse\;Confidence\;Bound}\;1/ε", ylabel = L"\textrm{Numbe\;of\;final\;clusters}")
for (i,e) in enumerate(ϵ_inv)
    res = HK_simulate_convergence(1/e,1/e; N = N, max_iter = max_iter, initial = initial)
    
    cluster = unique(res[2])
    number[i] = length(cluster)

    x = ones(length(cluster)) * e
    plt.scatter!(subplot = 1, x, (cluster .- 0.5) .*e, color =:red, marker = :diamond, markersize = 1.5)
    plt.scatter!(subplot = 2, 1 ./ x, (cluster .- 0.5), color =:red, marker = :diamond, markersize = 1.5)
end
#savefig("./Plots/HK-Phase-diag-sym-nonoise_N-$(N)_init-dist-$(initial).png")

# upper bound for limit of equal weight in all clusters
upper_bound = floor.(1/2 .* ϵ_inv)
plt.scatter!(subplot = 3, ϵ_inv, upper_bound, marker = :square, color = :grey, markersize = 2.5, )
#plt.plot!(subplot = 3, [1,max/2], [1,max/2], linestyle = :dash, color = :grey)

plt.scatter!(subplot = 3, ϵ_inv, number, color =:red, marker = :diamond, markersize = 2.5)

#plt.plot!(subplot = 2, xscale = :log)

##### Separate (for printing)
First top two plots, then last

In [ ]:
max = 30
ϵ_inv = 3:0.5:max

N = 300
max_iter = 2*N
initial = "uni"
number = zeros(length(ϵ_inv))

plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xscale = :log, xlabel = L"\textrm{Confidence\;Bound}\;ε", ylabel = L"\textrm{Final\;Opinion\;Cluster\;(shifted)}")

for (i,e) in enumerate(ϵ_inv)
    #simulate but stop if converged
    res = HK_simulate_convergence(1/e, 1/e; N = N, max_iter = max_iter, initial = initial)

    cluster = unique(res[2])

    x = ones(length(cluster)) * e
    #plt.scatter!(subplot = 1, x, (cluster .- 0.5) .*e, color =:red, marker = :diamond, markersize = 1.5)
    plt.scatter!(subplot = 1, 1 ./ x, (cluster .- 0.5), color =:red, marker = :diamond, markersize = 1.5)
end
#savefig("./Plots/HK-Phase-diag-sym-nonoise_N-$(N)_init-dist-$(initial).png")


plt.plot!()

In [ ]:
##### SAME AT COMPARE ICs ########################

min = 2
max = 45#0
step = 0.5
no_avg = 1#50
N = 300

ϵ_inv = min:step:max

max_iter = N*3
initial = "uni"
number = zeros(length(ϵ_inv))


for (i,e) in enumerate(ϵ_inv)
    avg = 0.
    for j in 1:no_avg
        res = HK_simulate_convergence(1/e, 1/e; N = N, max_iter = max_iter, initial = initial, seed = j)
        
        cluster = unique(res[2])
        #println(cluster)
        #print(length(cluster), " ")
        avg+=length(cluster)
    end

    number[i] = avg/no_avg
    println(i, " / ",length(ϵ_inv), " done: ", e)
end
#savefig("./Plots/HK-Phase-diag-sym-nonoise_N-$(N)_init-dist-$(initial).png")

print("Done!")

In [ ]:
#writedlm("HK-Cluster-nos-bounds-$(N)_init-dist-$(initial)_avg-$(no_avg).txt", number)

In [ ]:
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :bottomright, xlabel = L"\textrm{Inverse\;Confidence\;Bound}\;1/ε", ylabel = L"\textrm{Number\;of\;Final\;Clusters}")

# calculating soft upper bound for limit of equal weight in all clusters
upper_bound = floor.(1/2 .* ϵ_inv)
max_bound = floor(upper_bound[end])

# strict upper bound by averaging rule
plt.plot!(subplot = 1, min:max_bound, min:max_bound, color = :grey, linealpha = 1, linestyle = :dash, label=L"\textrm{Hard\;Bound}\;1/ε")
# soft upper bound
plt.plot!(subplot = 1, ϵ_inv, upper_bound, color = :grey, linealpha = 1,linetype=:steppost, label = L"\textrm{Soft\;Bound}\;\lfloor1/(2ε)\rfloor")

plt.scatter!(subplot = 1, ϵ_inv, number, color =:blue, marker = :x, markersize = 3, label = L"\textrm{Simulated}")
highlight = 8
plt.scatter!(subplot = 1, [ ϵ_inv[highlight] ], [ number[highlight] ], color =:red, marker = :x, markersize = 3, label = :none)
factor = 2.5
#plt.plot!(subplot = 1, ϵ_inv, 1/factor .* ϵ_inv, linecolor =:red, label = L"\textrm{Approximation\;}1/(%$factor ε)")
#savefig("./Plots/HK-Cluster-nos-bounds_N-$(N)_init-dist-$(initial)_avg-$(no_avg).png")

plt.plot!()

### Meta-Stability

In [ ]:
N = 25
ϵ = 0.2
max_iter = 99
initial = "uni"
result = HK_simulate(ϵ; N = N, max_iter = max_iter, initial = initial)

#for plotting purpose
max_iter = 30
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    #color = get(scheme, (i-1)/(N-1))
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 1, linesize = 1)#color = CList[i])
end
#savefig("./Plots/HK-metastability_N-$(N)_eps-$(ϵ)_init-dist-$(initial).png")

plt.plot!()

In [ ]:
x_i = collect(0:1/(N-1):1)
x_i2 = copy(x_i)

#Shifting 0.5 to 0.51
deleteat!(x_i, findall(x->x==0.5, x_i))
append!(x_i, [(0.51)])

#Weight 5 on 0.5
deleteat!(x_i2, findall(x->x==0.5, x_i))
append!(x_i2,zeros(5) .+(0.5))

sort!(x_i)
sort!(x_i2)

result = HK_simulate(x_i, ϵ, ϵ; max_iter = 99)
result2 = HK_simulate(x_i2, ϵ, ϵ; max_iter = 99)

print("Done")

In [ ]:
#for plotting purpose
max_iter = 7
plt.plot(layout = (1,2), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(subplot = 2, legend = :none, xlabel = L"\textrm{Iteration}\;t", ytick = :none)
highlight = 13
for i in 1:length(x_i)
    if i != highlight
        plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
    else
    end
end
for i in 1:length(x_i2)
    if i != highlight
        plt.plot!(subplot = 2, 0:max_iter,result2[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
    else
    end
end
plt.plot!(subplot = 1, 0:max_iter,result[highlight,1:max_iter+1], color =:red, colorbar = :false, linealpha = 0.5, linesize = 0.3)
plt.plot!(subplot = 2, 0:max_iter,result2[highlight,1:max_iter+1], color =:red, colorbar = :false, linealpha = 0.5, linesize = 0.3)


#savefig("./Plots/HK-metastability-fails_N-$(length(x_i))-$(length(x_i2))_eps-$(ϵ)_shifting-0.5_weight-5-on-0.5.png")

plt.plot!()

In [ ]:
N = 300
ϵ = 1/6
max_iter = 150

initial = "uni"
result = HK_simulate(ϵ; N = N, max_iter = max_iter, initial = initial)

#for plotting purpose
max_iter = 150
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    #color = get(scheme, (i-1)/(N-1))
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 1, linesize = 1)#color = CList[i])
end
#savefig("./Plots/HK-metastability_N-$(N)_eps-$(round(ϵ,digits = 4))_init-dist-$(initial).png")

plt.plot!()

### Compare ICs
(equidistant vs. uniformly random)

In [ ]:
min = 3
max = 100
step = 0.5
no_avg = 50#50
N = 600

ϵ_inv = min:step:max

max_iter = 1200
number_rand = zeros(length(ϵ_inv))
number_uni = zeros(length(ϵ_inv))


for (i,e) in enumerate(ϵ_inv)
    avg = 0.
    res_uni = HK_simulate_convergence(1/e, 1/e; N = N, max_iter = max_iter, initial = "uni")
    number_uni[i] = length(unique(res_uni[2]))
    for j in 1:no_avg
        res_rand = HK_simulate_convergence(1/e, 1/e; N = N, max_iter = max_iter, initial = "rand", seed = j)
        
        avg += length(unique(res_rand[2]))
    end

    number_rand[i] = avg/no_avg
    println(i, " / ",length(ϵ_inv), " done: ", e)
end


print("Done!")

In [ ]:
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :bottomright, xlabel = L"\textrm{Inverse\;Confidence\;Bound}\;1/ε", ylabel = L"\textrm{Number\;of\;Final\;Clusters}")
col = cgrad(:corkO, 10, categorical = true)

# strict upper bound by averaging rule
plt.plot!(subplot = 1, min:max_bound, min:max_bound, color = col[1], linealpha = 1, linestyle = :dash, label=L"\textrm{Hard\;Bound}\;1/ε")

# calculating soft upper bound for limit of equal weight in all clusters
upper_bound = floor.(1/2 .* ϵ_inv)
max_bound = floor(upper_bound[end])
plt.plot!(subplot = 1, ϵ_inv, upper_bound, color = col[10], linealpha = 0.5,linetype=:steppost, label = L"\textrm{Soft\;Bound}\;\lfloor1/(2ε)\rfloor")

# Simulations
plt.scatter!(subplot = 1, ϵ_inv, number_uni, color = col[3], marker = :+, markersize = 3, label = L"\textrm{Equidistant}")
plt.scatter!(subplot = 1, ϵ_inv, number_rand, color = col[7], marker = :x, markersize = 3, label = L"\sim \mathcal{U}[0,1]")

factor = 2.6
plt.plot!(subplot = 1, ϵ_inv, 1/factor .* ϵ_inv, linecolor =:red, linealpha = 0.5, label = L"\textrm{Approximation\;}1/(%$factor ε)")

#savefig("./Plots/HK-dist-clusters_comp-uni-vs-rand-IC_N-$(N).png")

plt.plot!()

### Asymmetric intervals

In [ ]:
N = 301
ϵ_l = 0.05
ϵ_r = 0.1
ϵ = (ϵ_l + ϵ_r)/2
max_iter = 99
initial = "uni"
result = HK_simulate(ϵ_l, ϵ_r; N = N, max_iter = max_iter, initial = initial)

#for plotting purpose
max_iter = 20
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    #color = get(scheme, (i-1)/(N-1))
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
end
#savefig("./Plots/HK-asym_N-$(N)_eps-L$(ϵ_l)-R$(ϵ_r)_init-dist-$(initial).png")
#savefig("./Plots/HK-dist-clusters_N-$(N)_eps-L$(ϵ_l)-R$(ϵ_r)_init-dist-$(initial).png")
plt.plot!()

### Noise
running noise script will massively increase notebook size. Therefore change output before saving to: print("Done")

In [ ]:
N = 35
#ϵ_l = 0.1
#ϵ_r = 0.1
ϵ = 0.1
noise_p = ϵ/2
#noise_f = gaussian_random_jumps
#noise_name = "gaussian-jumps"
noise_f = uniform_bounded_jumps
noise_name = "uni-bounded"

max_iter = 5000
initial = "rand"

result = HK_simulate(ϵ;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)

print("Done")


In [ ]:
#for plotting purpose
max_iter = 1000 
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    plt.plot!(subplot = 1, 1:max_iter+1,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
end

plt.plot!(xaxis=:log)
#savefig("./Plots/HK-noisy-xlog_N-$(N)_eps-$(ϵ)_noise-$(noise_name)-$(round(noise_p, digits = 3))_init-dist-$(initial).png")
plt.plot!()

# for better plotting, start counting at t = 1 for initial distribution instead of 0 !

print("Done")

In [ ]:
savefig("./Plots/HK-noisy-xlog_N-$(N)_eps-$(ϵ)_noise-$(noise_name)-$(round(noise_p, digits = 3))_init-dist-$(initial).png")

#### No quasi-consensus

In [ ]:
N = 10
#ϵ_l = 0.1
#ϵ_r = 0.1
ϵ = 0.01
noise_p = ϵ * 0.7
#noise_f = gaussian_random_jumps
#noise_name = "gaussian-jumps"
noise_f = uniform_bounded_jumps
noise_name = "uni-bounded"

max_iter = 9999
initial = "const"

result = HK_simulate(ϵ;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)

#for plotting purpose
max_iter = 5000
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi = 400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
for i in 1:N
    plt.plot!(subplot = 1, 1:max_iter+1,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
end

#savefig("./Plots/HK-noisy-no-quasi-cons_N-$(N)_eps-$(ϵ)_noise-$(noise_name)-$(round(noise_p, digits = 3))_init-dist-$(initial).png")
plt.plot!()#xaxis=:log)
# for better plotting, start counting at t = 1 for initial distribution instead of 0 !

print("Done")

### Variable confidence bound

#### Functions

In [ ]:
function epsilon_quadratic_amplify(x, min, max; middle = 0.5)
    epsilon = min + 4* (x-0.5)^2 * (max-min)
    return round(epsilon, digits = 9)
end
function epsilon_quadratic_attenuate(x, min, max; middle = 0.5)
    epsilon = - 4 * (max-min) * (x-0.5)^2 + max
    return round(epsilon, digits = 9)
end
function epsilon_quadratic(x::Number, min, mid, max; middle = 0.5)
    
    if x > middle
        ϵ_l = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
        ϵ_r = epsilon_quadratic_amplify(x, mid, max; middle = middle)
    else
        ϵ_l = epsilon_quadratic_amplify(x, mid, max; middle = middle)
        ϵ_r = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
    end

    return [ϵ_l, ϵ_r]
end

function epsilon_quadratic(x_vec::Vector, min, mid, max; middle = 0.5)
    ϵ_left = zeros(length(x_vec))
    ϵ_right = zeros(length(x_vec))

    for (i,x) in enumerate(x_vec)
        if x > middle
            ϵ_l = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
            ϵ_r = epsilon_quadratic_amplify(x, mid, max; middle = middle)
        else
            ϵ_l = epsilon_quadratic_amplify(x, mid, max; middle = middle)
            ϵ_r = epsilon_quadratic_attenuate(x, min, mid; middle = middle)
        end
        
        ϵ_left[i] = ϵ_l 
        ϵ_right[i] = ϵ_r
    end
    return [ϵ_left, ϵ_right ]
end

In [ ]:
min = 0.1
mid = 0.2
max = 0.3

point = [0.3,0.7,0.9, 0.99]
x = collect(0:0.01:1)

col = cgrad(:corkO, 10, categorical = true)

plt.plot(layout = (1,1), size =(500,300), fontsize = 16, margin=3mm, grid =:none, dpi = 200)
plt.plot!(xaxis = L"\textrm{Opinion\;}x_i", yaxis = L"ε_{l/r}(x_i)")#, title = "Quadratic ε (min $min, mid $mid, max $max)")

plt.plot!(x, epsilon_quadratic_amplify.(x,mid, max), ylim = (0,0.5), label = L"\textrm{Amplify}", color =:green)
plt.plot!(x, epsilon_quadratic_attenuate.(x,min, mid), ylim = (0,0.5), label = L"\textrm{Attenuate}", color =:red)

plt.scatter!(point, epsilon_quadratic(point, min, mid, max)[1], label = L"\textrm{Left\;bound}\;\varepsilon_l", color = col[4], marker = :ltriangle, ms = 9)
plt.scatter!(point, epsilon_quadratic(point, min, mid, max)[2], label = L"\textrm{Right\;bound}\;\varepsilon_l", color = col[8], marker = :rtriangle, ms = 9)

#savefig("./Plots/HK-quadratic-eps-values_N-$(N)_min-$min-mid-$mid-max-$(max).png")
plt.plot!()


In [ ]:
# for variable intervals
# only difference in calculation of I matrix (ϵ)

function HK_update_variable!(x_i, x_f, min, mid, max; err_tol = 1e-9, noise_function = no_jumps, noise_param = 0.)
    #x_i and x_f have to be distinct vectors and not pointers to the same entity
    N =  convert(Int,length(x_i))
    #Calculate interaction matrix
    
    I = Matrix{Float64}(LA.I, N, N)
    
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i[i]-x_i[j]) < epsilon_quadratic(x_i[j], min, mid, max)[1] +err_tol ### CHANGE
                I[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i[i]-x_i[j]) < epsilon_quadratic(x_i[i], min, mid, max)[2]+err_tol ### CHANGE
                I[i,j] = 1.0
            else
            end
        end
    end
    #normalize each row of the row-stochastic matrix
    for i in 1:N
        norm = sum(I[i,:])
        I[i,:] = I[i,:]./norm
    end
    
    #Generate noise if required
    ξ = noise_function(noise_param, N)   
    #CAREFUL: Nedd to use same noise if comparing plots (see below)


    #Update state vector
    for i in 1:N
        x_update = 0.

        # average over neigbouring opinions
        for j in 1:N
            if I[i,j]>err_tol
                x_update += I[i,j] * x_i[j] 
            else
            end
        end

        # add noise
        x_update += + ξ[i]
        
        # Adsorbing boundary condition
        if x_update > 1
            x_update = 1
        elseif x_update < 0
            x_update = 0
        else
        end
        
        #update final state vector
        x_f[i] = round(x_update, digits = 9)
    end
end

In [ ]:
# only difference is update function called
function HK_simulate_variable(min, mid, max;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0.)
    #Initial Distribution
    rng = MersenneTwister(2310)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end

    #Initialize Simulation
    trajectories = zeros(N,max_iter+1)
    trajectories[:,1] = x_i
    x_old = x_i
    x_new = copy(x_i)
    
    #Simulation
    for i in 1:max_iter
        HK_update_variable!(x_old,x_new, min, mid, max; noise_function = noise_function, noise_param = noise_param) ### CHANGE
        trajectories[:,i+1] = x_new
        x_old = copy(x_new)
    end
    return trajectories
end

In [ ]:
# for variable intervals
# only difference in calculation of I matrix (ϵ) AND updates both a const and variable ϵ with the same noise 

function HK_update_variable!(x_i, x_i_var, x_f, x_f_var, min, mid, max, ϵ; err_tol = 1e-9, noise_function = no_jumps, noise_param = 0.)
    #x_i and x_f have to be distinct vectors and not pointers to the same entity
    N =  convert(Int,length(x_i))

    #Calculate 'normal' interaction matrix
    I = Matrix{Float64}(LA.I, N, N)
    ϵ_l = ϵ
    ϵ_r = ϵ
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i[i]-x_i[j]) < ϵ_l+err_tol
                I[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i[i]-x_i[j]) < ϵ_r+err_tol
                I[i,j] = 1.0
            else
            end
        end
    end

    #Calculate variable interaction matrix
    I_var = Matrix{Float64}(LA.I, N, N)
    for i in 1:(N-1)
        for j in (i+1):N
            # if i (smaller) lies within the left bound of j (bigger), then i influences j
            if abs(x_i_var[i]-x_i_var[j]) < epsilon_quadratic(x_i_var[j], min, mid, max)[1] +err_tol ### CHANGE
                I_var[j,i] = 1.0
            else
            end
            # if j (bigger) lies within the right bound of i (smaller), then j influences i
            if abs(x_i_var[i]-x_i_var[j]) < epsilon_quadratic(x_i_var[i], min, mid, max)[2]+err_tol ### CHANGE
                I_var[i,j] = 1.0
            else
            end
        end
    end

    #normalize each row of the row-stochastic matrix
    for i in 1:N
        norm = sum(I[i,:])
        I[i,:] = I[i,:]./norm
    end
    for i in 1:N
        norm = sum(I_var[i,:])
        I_var[i,:] = I_var[i,:]./norm
    end

    #Generate noise if required
    ξ = noise_function(noise_param, N) 


    #Update state vector
    for i in 1:N
        x_update = 0.
        x_update_var = 0.

        # average over neigbouring opinions
        for j in 1:N
            if I[i,j]>err_tol
                x_update += I[i,j] * x_i[j] 
            else
            end
            if I_var[i,j]>err_tol
                x_update_var += I_var[i,j] * x_i_var[j] 
            else
            end
        end

        # add SAME noise in bothe cases
        x_update += ξ[i]
        x_update_var += ξ[i]
        
        # Adsorbing boundary condition
        if x_update > 1
            x_update = 1
        elseif x_update < 0
            x_update = 0
        else
        end
        # Adsorbing boundary condition (variable)
        if x_update_var > 1
            x_update_var = 1
        elseif x_update_var < 0
            x_update_var = 0
        else
        end
        
        #update final state vector
        x_f[i] = round(x_update, digits = 9)
        x_f_var[i] = round(x_update_var, digits = 9)
    end
end

In [ ]:
# only difference is update function called
function HK_simulate_variable(min, mid, max, ϵ;N = 100, max_iter = 20, initial = "uni", noise_function = no_jumps, noise_param = 0.)
    #Initial Distribution
    rng = MersenneTwister(2310)
    if initial == "rand"
        x_i = rand!(rng, zeros(N))
        sort!(x_i)#, rev=:true)
    elseif initial == "uni"
        x_i = collect(0:1/(N-1):1)
    elseif initial == "const"
        x_i = ones(N) .* 0.5
    else
        println("Choose either rand or uni or const for intial distribution")
        return -1
    end
    #println("State ",x_i)
    x_i_var = copy(x_i)

    #Initialize Simulation
    trajectories = zeros(N,max_iter+1)
    trajectories[:,1] = x_i
    x_old = copy(x_i)
    x_new = copy(x_i)
    trajectories_var = copy(trajectories)
    x_old_var = copy(x_i_var)
    x_new_var = copy(x_i_var)
    
    #Simulation
    for i in 1:max_iter
        HK_update_variable!(x_old, x_old_var, x_new, x_new_var, min, mid, max, ϵ; noise_function = noise_function, noise_param = noise_param)
        
        trajectories[:,i+1] = x_new
        trajectories_var[:,i+1] = x_new_var
        x_old = copy(x_new)
        x_old_var = copy(x_new_var)
    end
    return trajectories, trajectories_var
end

#### Simulation w/o noise

In [ ]:
N = 300

ϵ = 0.1

mid = ϵ
delta = 0.015
min = round(mid-delta, digits = 6)
max = round(mid+delta, digits = 6)

mid_2 = mid
delta_2 = 0.025
min_2 = round(mid_2-delta_2, digits = 6)
max_2 = round(mid_2 + delta_2, digits = 6)


noise_f = no_jumps
noise_name = "no"

max_iter = 99
initial = "uni"

result = HK_simulate(ϵ;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)
result_variable = HK_simulate_variable(min, mid, max;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)
result_variable_2 = HK_simulate_variable(min_2, mid_2, max_2;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)
#result, result_variable = HK_simulate_variable(min, mid, max, ϵ;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = 0)


#for plotting purpose
max_iter = 25
plt.plot(layout = (1,3), size =(900,400), fontsize = 10, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, ylabel = L"\textrm{Opinion}\;x_i(t)")#, title = L"\varepsilon = %$ϵ")
plt.plot!(subplot = 2,  title = L"p_1=(%$min,\; %$mid, \;%$max)")#, ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(subplot = 3, title = L"p_2=(%$min_2,\; %$mid_2, \;%$max_2)")#, ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(xticks = [0,10,20], xlabel = L"\textrm{Iteration}\;t", legend = :none)
for i in 1:N
    plt.plot!(subplot = 1, 0:max_iter,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
    plt.plot!(subplot = 2, 0:max_iter,result_variable[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
    plt.plot!(subplot = 3, 0:max_iter,result_variable_2[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
end

#savefig("./Plots/HK-quadratic_N-$(N)_eps-$(ϵ)_delta-$(delta)-$(delta_2)_noise-$(noise_name)_init-dist-$(initial).png")
plt.plot!()


In [ ]:
#res = result; delt = 0
#res = result_variable; delt = delta
res = result_variable_2; delt = delta_2

#for plotting purpose
max_iter = 25
plt.plot(layout = (1,1), size =(500,400), fontsize = 10, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(xticks = [0,5,10,15,20,25])

for i in 1:N
    #plt.plot!(subplot = 1, 0:max_iter,res[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
    plt.plot!(subplot = 1, 0:max_iter,result_variable[i,1:max_iter+1], color =:red, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
    plt.plot!(subplot = 1, 0:max_iter,result_variable_2[i,1:max_iter+1], color =:blue, colorbar = :false, linealpha = 0.5, linesize = 0.3)
end
#savefig("./Plots/HK-quadratic_N-$(N)_eps-$(ϵ)_delta-$(delt)_noise-$(noise_name)_init-dist-$(initial).png")
#savefig("./Plots/HK-quadratic-comp_N-$(N)_eps-$(ϵ)_noise-$(noise_name)_init-dist-$(initial).png")

plt.plot!()


#### Simulation w. noise

In [ ]:
N = 300

mid = 0.1
delta = 0.02
min = mid-delta
max = mid+delta
ϵ = mid
noise_p = max / 2
#noise_f = gaussian_random_jumps
#noise_name = "gaussian-jumps"
noise_f = uniform_bounded_jumps
noise_name = "uni-bounded"
#noise_f = no_jumps
#noise_name = "no"

max_iter = 999
initial = "uni"

result, result_variable = HK_simulate_variable(min, mid, max, ϵ;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)


#for plotting purpose
max_iter = 999
plt.plot(layout = (1,2), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(subplot = 2, legend = :none, xlabel = L"\textrm{Iteration}\;t")#, ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(xaxis =:log)
for i in 1:N
    plt.plot!(subplot = 1, 1:max_iter+1,result[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
    plt.plot!(subplot = 2, 1:max_iter+1,result_variable[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)
end

#savefig("./Plots/HK-quadratic_N-$(N)_eps-$(ϵ)_min-$min-mid-$mid-max-$(max)_noise-$(noise_name)-$(round(noise_p, digits = 3))_init-dist-$(initial).png")
plt.plot!()


In [ ]:
N = 300

min = 0.05
mid = 0.1
max = 0.15
ϵ = mid
noise_p = max / 4
#noise_f = gaussian_random_jumps
#noise_name = "gaussian-jumps"
noise_f = uniform_bounded_jumps
noise_name = "uni-bounded"
# noise_f = no_jumps
# noise_name = "no"

max_iter = 999
initial = "uni"

result_variable_noise = HK_simulate_variable(min, mid, max;N = N, max_iter = max_iter, initial = initial, noise_function = noise_f, noise_param = noise_p)


In [ ]:
#for plotting purpose
max_iter = 999
plt.plot(layout = (1,1), size =(500,400), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :none, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")

for i in 1:N
    #color = get(scheme, (i-1)/(N-1))
    plt.plot!(subplot = 1, 1:max_iter+1,result_variable_noise[i,1:max_iter+1], color =:corkO, line_z = i, colorbar = :false, linealpha = 0.5, linesize = 0.3)#color = CList[i])
end

#savefig("./Plots/HK-quadratic_N-$(N)_eps-$(ϵ)_min-$min-mid-$mid-max-$(max)_noise-$(noise_name)-$(round(noise_p, digits = 3))_init-dist-$(initial).png")
plt.plot!(xaxis = :log)

## Other Figures for Essay

In [ ]:
plt.plot(layout = (1,1), size =(200,200), fontsize = 16, margin=3mm, grid =:none, dpi=400, xlabel = L"ε_l", ylabel = L"ε_r")
plt.plot!([0,1],[0,1], arrow = true, color = :grey, alpha = 0.2)
plt.plot!(legend = :none, xticks=0:1, yticks=0:1)

plt.scatter!([0.05,0.1, 0.5, 0.7],[0.1,0.2,0.3, 0.9], color = :black, marker =:x, )

#savefig("./Plots/eps_param_space_diag.png")
savefig("./Plots/eps_param_space_off-diag.png")

plt.plot!()

In [ ]:
A = [1/2 1/2 0; 1/4 3/4 0; 0 0 1]
#A = [1/2 1/2 0; 1/3 1/3 1/3; 0 1/2 1/2]

xi = [0.9,0.45,0]
xi_2 = [0.6,0.15,0.1]

A[1,3]

In [ ]:
function A_predicted(x::Vector)
    return (x[1] + x[2] * 3/2 + x[3]) * 2/7
end
A_predicted(xi_2)

In [ ]:
max_iteration = 15
result = zeros(3,max_iteration+1)
result_2 = zeros(3,max_iteration+1)
result[:,1] = xi
result_2[:,1] = xi_2
x = copy(xi)
x_2 = copy(xi_2)
for t in 1:max_iteration
    x_new = zeros(3)
    x_2_new = zeros(3)
    for i in 1:3
        x_new[i] = x[1]*A[i,1]+x[2]*A[i,2]+x[3]*A[i,3]
        x_2_new[i] = x_2[1]*A[i,1]+x_2[2]*A[i,2]+x_2[3]*A[i,3]
    end
    x = copy(x_new)
    x_2 = copy(x_2_new)
    result[:,t+1] = x
    result_2[:,t+1] = x_2
end

plt.plot(layout = (1,1), size =(500,300), fontsize = 16, margin=3mm, grid =:none, dpi=400)
plt.plot!(subplot = 1, legend = :topright,  ncol=2, xlabel = L"\textrm{Iteration}\;t", ylabel = L"\textrm{Opinion}\;x_i(t)")
plt.plot!(ylim = [0,1])
col = cgrad(:corkO, 10, categorical = true)

#plt.plot!([0,max_iteration], [A_predicted(xi), A_predicted(xi)],  label = L"\textrm{Predicted}",c = col[8], lw = 5, alpha = 0.2)
#plt.plot!([0,max_iteration], [A_predicted(xi_2), A_predicted(xi_2)],  label = :none,c = col[8], lw = 5, alpha = 0.2)

plt.plot!(0:max_iteration,result[1,:], label = L"\textrm{A}", marker =:diamond, c = col[2], ls = :dot)
plt.plot!(0:max_iteration,result[2,:], label = L"\textrm{B}", marker =:diamond, c = col[4], ls = :dot)
plt.plot!(0:max_iteration,result[3,:], label = L"\textrm{C}", marker =:diamond, c = col[6], ls = :dot)

plt.plot!(0:max_iteration,result_2[1,:], label = L"\textrm{A}", marker =:x, ls = :dash, c = col[2])
plt.plot!(0:max_iteration,result_2[2,:], label = L"\textrm{B}", marker =:x, ls = :dash, c = col[4])
plt.plot!(0:max_iteration,result_2[3,:], label = L"\textrm{C}", marker =:x, ls = :dash, c = col[6])

#savefig("./Plots/DeGroot-updating.png")
print("Done")

